# DAHYUN+DARWIN = DAHWIN

In [ ]:
import torch
from torch import nn
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from typing import Tuple ,Dict,List
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
image_path = 'dawin_data'
image_path


In [ ]:
import os


# Get list of all files and directories in image_path
all_files = os.listdir(image_path)

# Loop through the list and print only directories
for file in all_files:
    if os.path.isdir(os.path.join(image_path, file)):
        print(file)


In [ ]:
import zipfile
from pathlib import Path
with zipfile.ZipFile('twice/dawin_data.zip','r') as zip_ref:
  print('Unzipping data')
  zip_ref.extractall()

In [ ]:
import pathlib
image_path = pathlib.Path(image_path)  
# Setup train and testing paths
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

In [ ]:
import random
from PIL import Image

# Set seed
# random.seed(42) # <- try changing this and see what happens

# 1. Get all image paths (* means "any combination")

image_path_list = list(image_path.glob("*/*/*.jpg")) + list(image_path.glob("*/*/*.png"))


# 2. Get random image path
random_image_path = random.choice(image_path_list)

# 3. Get image class from path name (the image class is the name of the directory where the image is stored)
image_class = random_image_path.parent.stem

# 4. Open image
img = Image.open(random_image_path)

# 5. Print metadata
print(f"Random image path: {random_image_path}")
print(f"Image class: {image_class}")
print(f"Image height: {img.height}") 

print(f"Image width: {img.width}")
img

In [ ]:
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import datasets,transforms

In [ ]:
data_transform = transforms.Compose([
    transforms.Resize(size=(224,224)),
    
    transforms.ToTensor()
])
data_transform_test = transforms.Compose([
    transforms.Resize(size=(224,224)),
    
    transforms.ToTensor()
])

In [ ]:
def plot_transformed_images(image_path_list, transformed_data, n, seed=32):
    """Plots a series of random images from image_paths.

    Will open n image paths from image_paths, transform them
    with transform and plot them side by side.

    Args:
        image_paths (list): List of target image paths. 
        transform (PyTorch Transforms): Transforms to apply to images.
        n (int, optional): Number of images to plot. Defaults to 3.
        seed (int, optional): Random seed for the random generator. Defaults to 42.
    """
    # random.seed(seed)
    random_image_paths = random.sample(image_path_list, k=n)
    for image_path in random_image_paths:
        with Image.open(image_path) as f:
            fig, ax = plt.subplots(1, 2)
            ax[0].imshow(f) 
            ax[0].set_title(f"Original \nSize: {f.size}")
            ax[0].axis("off")

            # Transform and plot image
            # Note: permute() will change shape of image to suit matplotlib 
            # (PyTorch default is [C, H, W] but Matplotlib is [H, W, C])
            transformed_image = transformed_data(f).permute(1, 2, 0) 
            ax[1].imshow(transformed_image) 
            ax[1].set_title(f"Transformed \nSize: {transformed_image.shape}")
            ax[1].axis("off")

            fig.suptitle(f"Class: {image_path.parent.stem}", fontsize=16)

In [ ]:
plot_transformed_images(image_path_list,data_transform,n=4)

In [ ]:
train_data = datasets.ImageFolder(root=train_dir,transform=data_transform,target_transform=None)
test_data = datasets.ImageFolder(root=test_dir,transform=data_transform)
print(f"Train data:\n{train_data} Test data \n{test_data}")

In [ ]:
class_names = train_data.classes

In [ ]:
class_names

In [ ]:
import os
from torch.utils.data import DataLoader
NUM_WORKERS = os.cpu_count()
train_dataloader = DataLoader(dataset=train_data,batch_size=1,num_workers=NUM_WORKERS,shuffle=False)
test_dataloader = DataLoader(dataset=test_data,batch_size=1,num_workers=NUM_WORKERS,shuffle=False)
train_dataloader,test_dataloader

In [ ]:
class DahwinTinyVGG(nn.Module):
   
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, 
                      out_channels=hidden_units, 
                      kernel_size=3, # how big is the square that's going over the image?
                      stride=1, # default
                      padding=1), # options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number 
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2) # default stride value is same as kernel_size
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            # Where did this in_features shape come from? 
            # It's because each layer of our network compresses and changes the shape of our inputs data.
            nn.Linear(in_features=hidden_units*56*56,
                      out_features=output_shape)
        )
    
    def forward(self, x: torch.Tensor):
        x = self.conv_block_1(x)
        # print(x.shape)
        x = self.conv_block_2(x)
        # print(x.shape)
        x = self.classifier(x)
        # print(x.shape)
        return x
        # return self.classifier(self.conv_block_2(self.conv_block_1(x))) # <- leverage the benefits of operator fusion

torch.manual_seed(42)
model_0 = DahwinTinyVGG(input_shape=3, # number of color channels (3 for RGB) 
                  hidden_units=10, 
                  output_shape=len(train_data.classes)).to(device)
model_0

In [ ]:
pip install torchinfo

In [ ]:
from torchinfo import summary
summary(model_0,input_size = [1,3,224,224])

In [ ]:
def train_step(model:torch.nn.Module,dataloader:torch.utils.data.DataLoader,loss_fn:torch.nn.Module,optimizer:torch.optim.Optimizer):
  model.train()
  train_loss,train_acc = 0,0
  for batch ,(x,y) in enumerate(dataloader):
    x,y = x.to(device),y.to(device)
    y_pred = model(x)
    loss = loss_fn(y_pred,y)
    train_loss += loss.item()
    loss.backward()
    optimizer.step()
    y_pred_class = torch.argmax(torch.softmax(y_pred,dim=1),dim=1)
    train_acc += (y_pred_class==y).sum().item()/len(y_pred)

  train_loss = train_loss/len(dataloader)
  train_acc = train_acc/len(dataloader)
  return train_loss,train_acc

In [ ]:
def test_step(model:torch.nn.Module,
              dataloader:torch.utils.data.DataLoader,
              loss_fn:torch.nn.Module):
  model.eval()
  test_loss,test_acc =0,0
  with torch.inference_mode():
    for batch ,(X,y) in enumerate(dataloader):
      X,y = X.to(device),y.to(device)
      test_pred_logits = model(X)
      loss = loss_fn(test_pred_logits,y)
      test_loss += loss.item()
      test_pred_labels = test_pred_logits.argmax(dim=1)
      test_acc += ((test_pred_labels==y).sum().item()/len(test_pred_labels))
  test_loss = test_loss/len(dataloader)
  test_acc = test_acc/len(dataloader)
  return test_loss,test_acc
     

In [ ]:
from tqdm.auto import tqdm
def train(model:torch.nn.Module,train_dataloader:torch.utils.data.DataLoader,test_dataloader:torch.utils.data.DataLoader,optimizer:torch.optim.Optimizer,loss_fn:torch.nn.Module=nn.CrossEntropyLoss(),epochs:int=5):
  results = {'train_loss':[],
             'train_acc':[],
             'test_loss':[],
             'test_acc':[]}
  for epoch in tqdm(range(epochs)):
    train_loss ,train_acc = train_step(model=model,dataloader=train_dataloader,loss_fn=loss_fn,optimizer=optimizer)
    test_loss,test_acc = test_step(model=model,dataloader=test_dataloader,loss_fn=loss_fn)
    print(f'{epoch} | Train loss:{train_loss:.4f} |Train acc:{train_acc:.4f} | Test loss: {test_loss:.4f} | Test acc: {test_acc:.4f}')
    results['train_loss'].append(train_loss)
    results['train_acc'].append(train_acc)
    results['test_loss'].append(test_loss)
    results['test_acc'].append(test_acc)
  return results

In [ ]:
# Set random seeds
torch.manual_seed(42) 
torch.cuda.manual_seed(42)

# Set number of epochs
NUM_EPOCHS = 5

# Recreate an instance of TinyVGG
model_0 = DahwinTinyVGG(input_shape=3, # number of color channels (3 for RGB) 
                  hidden_units=10, 
                  
                  output_shape=len(train_data.classes)).to(device)

# Setup loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(), lr=0.001)

# Start the timer
from timeit import default_timer as timer 
start_time = timer()

# Train model_0 
model_0_results = train(model=model_0, 
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn, 
                        epochs=NUM_EPOCHS,
                        )

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

In [ ]:
# Download custom image
import requests

# Setup custom image path
custom_image_path = pathlib.Path("dawin_data/ddubu.png")
with open(custom_image_path, "wb") as f:
        # When downloading from GitHub, need to use the "raw" file link
        # request = requests.get("https://i.pinimg.com/564x/d6/46/2f/d6462fe5457ba4df288f859e9eaa2195.jpg")
        request = requests.get("https://www.pinkvilla.com/files/dahyun_main.jpg")
        print(f"Downloading {custom_image_path}...")
        f.write(request.content)
# # Download the image if it doesn't already exist
# if not custom_image_path.is_file():
#     with open(custom_image_path, "wb") as f:
#         # When downloading from GitHub, need to use the "raw" file link
#         # request = requests.get("https://i.pinimg.com/564x/d6/46/2f/d6462fe5457ba4df288f859e9eaa2195.jpg")
#         request = requests.get("https://i.pinimg.com/564x/4e/21/d0/4e21d0b46f7f90183afe0a9dec394e11.jpg")
#         print(f"Downloading {custom_image_path}...")
#         f.write(request.content)
# else:
#     print(f"{custom_image_path} already exists, skipping download.")

In [ ]:
import torchvision
custom_image_uint8 = torchvision.io.read_image(str(custom_image_path)).type(torch.float32)
custom_image_uint8 = custom_image_uint8 / 255. 
# Print out image data
print(f"Custom image tensor:\n{custom_image_uint8}\n")
print(f"Custom image shape: {custom_image_uint8.shape}\n")
print(f"Custom image dtype: {custom_image_uint8.dtype}")

In [ ]:
# Load in custom image and convert the tensor values to float32
custom_image = torchvision.io.read_image(str(custom_image_path)).type(torch.float32)

# Divide the image pixel values by 255 to get them between [0, 1]
custom_image = custom_image / 255. 

# Print out image data
print(f"Custom image tensor:\n{custom_image}\n")
print(f"Custom image shape: {custom_image.shape}\n")
print(f"Custom image dtype: {custom_image.dtype}")

In [ ]:
plt.imshow(custom_image.permute(1, 2, 0))

In [ ]:
custom_image_transform =  transforms.Compose([
    transforms.Resize((224,224))
])
custom_image_transformed = custom_image_transform(custom_image)
print(f"Original shape:{custom_image.shape}")
print(f"New shape:{custom_image_transformed.shape}")

In [ ]:
plt.imshow(custom_image_transformed.permute(1, 2, 0))

In [ ]:
model_0.eval()
with torch.inference_mode():
  custom_image_pred = model_0(custom_image_transformed.unsqueeze(0).to(device))
custom_image_pred

In [ ]:
def pred_and_plot_image(model: torch.nn.Module, 
                        image_path: str, 
                        class_names: List[str] = None, 
                        transform=None,
                        device: torch.device = device):
    """Makes a prediction on a target image and plots the image with its prediction."""
    
    # 1. Load in image and convert the tensor values to float32
    target_image = torchvision.io.read_image(str(image_path)).type(torch.float32)
    
    # 2. Divide the image pixel values by 255 to get them between [0, 1]
    target_image = target_image / 255. 
    
    # 3. Transform if necessary
    if transform:
        target_image = transform(target_image)
    
    # 4. Make sure the model is on the target device
    model.to(device)
    
    # 5. Turn on model evaluation mode and inference mode
    model.eval()
    with torch.inference_mode():
        # Add an extra dimension to the image
        target_image = target_image.unsqueeze(dim=0)
    
        # Make a prediction on image with an extra dimension and send it to the target device
        target_image_pred = model(target_image.to(device))
        
    # 6. Convert logits -> prediction probabilities (using torch.softmax() for multi-class classification)
    target_image_pred_probs = torch.softmax(target_image_pred, dim=1)

    # 7. Convert prediction probabilities -> prediction labels
    target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)
    
    # 8. Plot the image alongside the prediction and prediction probability
    plt.imshow(target_image.squeeze().permute(1, 2, 0)) # make sure it's the right size for matplotlib
    if class_names:
        title = f"Pred: {class_names[target_image_pred_label.cpu()]} | Prob: {target_image_pred_probs.max().cpu():.3f}"
    else: 
      title = f"Pred: {target_image_pred_label} | Prob: {target_image_pred_probs.max().cpu():.3f}"
    plt.title(title)
    plt.axis(False);

# MY FIRST AI MODEL WITH DAHYUN


In [ ]:
# Pred on our custom image
pred_and_plot_image(model=model_0,
                    image_path=custom_image_path,
                    class_names=class_names,
                    transform=custom_image_transform,
                    device=device)

In [ ]:
train_data.classes